In [1]:
import pandas as pd
import json
import os
import numpy as np
import re #regex 

In [2]:
dfs = []
for r, d, f in os.walk('input'):
    for file in f:
        if 'plus_one_control.json' in file:  # alt: if ‘control' in file:
            dfs.append(pd.read_json('%s/%s' % (r, file), lines=True))

df_cen = pd.concat(dfs)
#df_cen = df_cen.dropna(subset=['withheld_in_countries'])

In [3]:
worthKeeping = ["text", "truncated", "user",
                "withheld_in_countries", "entities", "lang",
                "possibly_sensitive", "extended_tweet"]
df_cen = df_cen[worthKeeping]

In [4]:
df_cen

,text,truncated,user,withheld_in_countries,entities,lang,possibly_sensitive,extended_tweet
0,文i學i界イベとと～こくとの回想、かたいくんのCVが梶裕貴で本当に良かったと思った瞬間だったよ,False,"{'id': 1263015148366643200, 'id_str': '1263015...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,NaN,NaN
1,RT @Roi_Mobile: 和製さんは音楽botとエロイプするのが趣味らしいです htt...,False,"{'id': 1354706858636808193, 'id_str': '1354706...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,0.0,NaN
2,@tqriir #الا_رسول_الله_ﷺ \n#مقاطعه_المنتجات_ال...,False,"{'id': 2958543338, 'id_str': '2958543338', 'na...",NaN,"{'hashtags': [{'text': 'الا_رسول_الله_ﷺ', 'ind...",und,NaN,NaN
3,RT @adam_lovex: Who’s trynna be the Hinata to ...,False,"{'id': 1251080089116106752, 'id_str': '1251080...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
4,3 people followed me // automatically checked ...,False,"{'id': 283124572, 'id_str': '283124572', 'name...",NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",en,0.0,NaN
...,...,...,...,...,...,...,...,...
178,RT @Mira_n420: 私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなん...,False,"{'id': 3232626306, 'id_str': '3232626306', 'na...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,NaN,NaN
179,RT @minombreeschris: todo el mundo manda foto ...,False,"{'id': 1267532670562701312, 'id_str': '1267532...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",es,0.0,NaN
180,@iiSpez درر,False,"{'id': 353999477, 'id_str': '353999477', 'name...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ar,NaN,NaN
181,ja participo des do ano passado,False,"{'id': 1092518124455448576, 'id_str': '1092518...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",pt,NaN,NaN


In [5]:
#replace the NaN with coherent values to make further processing easier
df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)
df_cen

,text,truncated,user,withheld_in_countries,entities,lang,possibly_sensitive,extended_tweet
0,文i學i界イベとと～こくとの回想、かたいくんのCVが梶裕貴で本当に良かったと思った瞬間だったよ,False,"{'id': 1263015148366643200, 'id_str': '1263015...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,0.0,NaN
1,RT @Roi_Mobile: 和製さんは音楽botとエロイプするのが趣味らしいです htt...,False,"{'id': 1354706858636808193, 'id_str': '1354706...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,0.0,NaN
2,@tqriir #الا_رسول_الله_ﷺ \n#مقاطعه_المنتجات_ال...,False,"{'id': 2958543338, 'id_str': '2958543338', 'na...",NaN,"{'hashtags': [{'text': 'الا_رسول_الله_ﷺ', 'ind...",und,0.0,NaN
3,RT @adam_lovex: Who’s trynna be the Hinata to ...,False,"{'id': 1251080089116106752, 'id_str': '1251080...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
4,3 people followed me // automatically checked ...,False,"{'id': 283124572, 'id_str': '283124572', 'name...",NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",en,0.0,NaN
...,...,...,...,...,...,...,...,...
178,RT @Mira_n420: 私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなん...,False,"{'id': 3232626306, 'id_str': '3232626306', 'na...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,0.0,NaN
179,RT @minombreeschris: todo el mundo manda foto ...,False,"{'id': 1267532670562701312, 'id_str': '1267532...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",es,0.0,NaN
180,@iiSpez درر,False,"{'id': 353999477, 'id_str': '353999477', 'name...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ar,0.0,NaN
181,ja participo des do ano passado,False,"{'id': 1092518124455448576, 'id_str': '1092518...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",pt,0.0,NaN


In [6]:
#recover the full text for truncated tweets

dfRaw = df_cen.values
for line in dfRaw:
    if not pd.isna(line[-1]):
        line[0] = line[-1]["full_text"]
        
    line[0] = re.sub(r'http\S+', '', line[0])
    #remove urls from tweets
    #they are shortened anyway so we can't make use of them

dfRaw = np.delete(dfRaw, len(worthKeeping)-1, axis=1) #remove "extended_tweet"
worthKeeping.remove("extended_tweet")

dfRaw = np.delete(dfRaw, 1, axis=1) #remove "truncated"
worthKeeping.remove("truncated")

In [7]:
#extract hashtags seperately

for line in dfRaw:
    line[3] = [x["text"] for x in line[3]["hashtags"]]
worthKeeping[3] = "hashtags"

In [8]:
#create a feature for user-verified and user-followers_count

verified = [line[1]["verified"] for line in dfRaw]
followers = [line[1]["followers_count"] for line in dfRaw]
location = [line[1]["location"] for line in dfRaw]

dfRaw = np.c_[dfRaw, verified, followers, location]
worthKeeping += ["verified_account", "followers_count", "location"]

In [9]:
#binary feature for whether the tweet has been withheld anywhere

withheld = []
for line in dfRaw:
    if not isinstance(line[2], list):
        line[2] = []
    withheld.append(len(line[2]) != 0)
        
dfRaw = np.c_[dfRaw, withheld]
worthKeeping += ["withheld_anywhere"]

In [10]:
df_cen = pd.DataFrame(dfRaw, columns = worthKeeping)
df_cen

,text,user,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,withheld_anywhere
0,文i學i界イベとと～こくとの回想、かたいくんのCVが梶裕貴で本当に良かったと思った瞬間だったよ,"{'id': 1263015148366643200, 'id_str': '1263015...",[],[],ja,0.0,False,258,シマエナガのいる図書館,False
1,RT @Roi_Mobile: 和製さんは音楽botとエロイプするのが趣味らしいです,"{'id': 1354706858636808193, 'id_str': '1354706...",[],[],ja,0.0,False,61,None,False
2,@tqriir #الا_رسول_الله_ﷺ \n#مقاطعه_المنتجات_ال...,"{'id': 2958543338, 'id_str': '2958543338', 'na...",[],"[الا_رسول_الله_ﷺ, مقاطعه_المنتجات_الفرنسيه155]",und,0.0,False,370,Hashemite Kingdom of Jordan,False
3,RT @adam_lovex: Who’s trynna be the Hinata to ...,"{'id': 1251080089116106752, 'id_str': '1251080...",[],[],en,0.0,False,846,Melbourne,False
4,3 people followed me // automatically checked by,"{'id': 283124572, 'id_str': '283124572', 'name...",[],[],en,0.0,False,1281,None,False
...,...,...,...,...,...,...,...,...,...,...
28560,RT @Mira_n420: 私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなん...,"{'id': 3232626306, 'id_str': '3232626306', 'na...",[],[],ja,0.0,False,316,南信州の山の中,False
28561,RT @minombreeschris: todo el mundo manda foto ...,"{'id': 1267532670562701312, 'id_str': '1267532...",[],[],es,0.0,False,178,None,False
28562,@iiSpez درر,"{'id': 353999477, 'id_str': '353999477', 'name...",[],[],ar,0.0,False,13770,None,False
28563,ja participo des do ano passado,"{'id': 1092518124455448576, 'id_str': '1092518...",[],[],pt,0.0,False,404,greed Island,False


In [11]:
cleanCols = filter(lambda x: x != "user", worthKeeping)
df_clean = df_cen[cleanCols]

In [12]:
df_clean

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,withheld_anywhere
0,文i學i界イベとと～こくとの回想、かたいくんのCVが梶裕貴で本当に良かったと思った瞬間だったよ,[],[],ja,0.0,False,258,シマエナガのいる図書館,False
1,RT @Roi_Mobile: 和製さんは音楽botとエロイプするのが趣味らしいです,[],[],ja,0.0,False,61,None,False
2,@tqriir #الا_رسول_الله_ﷺ \n#مقاطعه_المنتجات_ال...,[],"[الا_رسول_الله_ﷺ, مقاطعه_المنتجات_الفرنسيه155]",und,0.0,False,370,Hashemite Kingdom of Jordan,False
3,RT @adam_lovex: Who’s trynna be the Hinata to ...,[],[],en,0.0,False,846,Melbourne,False
4,3 people followed me // automatically checked by,[],[],en,0.0,False,1281,None,False
...,...,...,...,...,...,...,...,...,...
28560,RT @Mira_n420: 私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなん...,[],[],ja,0.0,False,316,南信州の山の中,False
28561,RT @minombreeschris: todo el mundo manda foto ...,[],[],es,0.0,False,178,None,False
28562,@iiSpez درر,[],[],ar,0.0,False,13770,None,False
28563,ja participo des do ano passado,[],[],pt,0.0,False,404,greed Island,False


In [13]:
df_clean["withheld_anywhere"].describe()

count     28565
unique        2
top       False
freq      28559
Name: withheld_anywhere, dtype: object